In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Pollen-Adaptation/cyclegan

Mounted at /content/drive
/content/drive/My Drive/Pollen-Adaptation/cyclegan


In [2]:
## install section for Colab

! pip install dominate
! pip install visdom



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 676 kB 11.1 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=9d7a33c1b55e7134f049edd87f2969ce476bb8126f5f9142349541479b7038dd
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha256=9b7a49dc576cd9dd147b4d8a7abdb19b5d175c1125eccfb51911b2fa059a0504
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom torchfile


In [ ]:
import time
import argparse
import torch
import os
from data import CreateDataLoader
from models import create_model
from util.visualizer import Visualizer
from options.train_options import TrainOptions
torch.cuda.is_available()

True

In [ ]:
# python train.py --name cycada_svhn2mnist_noIdentity \
#     --resize_or_crop=None \
#     --loadSize=32 --fineSize=32 --which_model_netD n_layers --n_layers_D 3 \
#     --model cycle_gan_semantic \
#     --lambda_A 1 --lambda_B 1 --lambda_identity 0 \
#     --no_flip --batchSize 100 \
#     --dataset_mode mnist_svhn --dataroot /x/jhoffman/ \
#     --which_direction BtoA

In [ ]:
### Loading parameters configuration
opt = TrainOptions()
opt.initialize()
trainOPT = opt.parser.parse_args(args=[ '--name', 'cycada_svhn2mnist_noIdentity',
                                       '--resize_or_crop', 'None',
                                       '--loadSize', '32',
                                       '--fineSize', '32',
                                       '--which_model_netD', 'n_layers',
                                       '--n_layers_D', '3',
                                       '--model', 'cycle_gan',
                                       '--lambda_A', '1',
                                       '--lambda_B', '1',
                                       '--lambda_identity', '0',
                                       '--no_flip',
                                       '--batchSize', '100',
                                       '--dataset_mode', 'mnist_svhn',
                                       '--which_direction', 'AtoB',
                                       '--dataroot', '/content/drive/My Drive/Pollen-Adaptation/cyclegan/dataset',
                                        ])
trainOPT.isTrain = True
trainOPT.gpu_ids = [int(trainOPT.gpu_ids)]
print(trainOPT.gpu_ids)

##

[0]


In [ ]:
##  Loading Data

data_loader = CreateDataLoader(trainOPT)

dataset = data_loader.load_data()
dataset_size = len(data_loader)
print('#training images = %d' % dataset_size)


CustomDatasetDataLoader
dataset [MnistSvhnDataset] was created
Namespace(batchSize=100, beta1=0.5, checkpoints_dir='./checkpoints', continue_train=False, dataroot='/content/drive/My Drive/Pollen-Adaptation/cyclegan/dataset', dataset_mode='mnist_svhn', display_freq=400, display_id=1, display_ncols=4, display_port=8097, display_server='http://localhost', display_winsize=256, epoch_count=1, fineSize=32, gpu_ids=[0], init_type='normal', input_nc=3, isTrain=True, lambda_A=1.0, lambda_B=1.0, lambda_identity=0.0, loadSize=32, lr=0.0002, lr_decay_iters=50, lr_policy='lambda', max_dataset_size=inf, model='cycle_gan', nThreads=4, n_layers_D=3, name='cycada_svhn2mnist_noIdentity', ndf=64, ngf=64, niter=100, niter_decay=100, no_dropout=False, no_flip=True, no_html=False, no_lsgan=False, norm='instance', output_nc=3, phase='train', pool_size=50, print_freq=100, resize_or_crop='None', save_epoch_freq=5, save_latest_freq=5000, serial_batches=False, suffix='', update_html_freq=1000, verbose=False, whi

In [ ]:
## initialize model

model = create_model(trainOPT)
model.setup(trainOPT)


initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
model [CycleGANModel] was created
---------- Networks initialized -------------
[Network G_A] Total number of parameters : 11.378 M
[Network G_B] Total number of parameters : 11.378 M
[Network D_A] Total number of parameters : 2.765 M
[Network D_B] Total number of parameters : 2.765 M
-----------------------------------------------


In [ ]:
## visualizer
#! python -m visdom.server

visualizer = Visualizer(trainOPT)
log_filename = os.path.join('/content/drive/My Drive/Pollen-Adaptation/cyclegan/checkpoints','train.log')
print(log_filename)

Setting up a new session...


create web directory ./checkpoints/cycada_svhn2mnist_noIdentity/web...
/content/drive/My Drive/Pollen-Adaptation/cyclegan/checkpoints/train.log


In [ ]:

total_steps = 0


##log_filename = os.path.join(trainOPT.checkpoints_dir,'train.log')    

for epoch in range(trainOPT.epoch_count, trainOPT.niter + trainOPT.niter_decay + 1):
    epoch_start_time = time.time()
    iter_data_time = time.time()
    epoch_iter = 0

    for i, data in enumerate(dataset):
        iter_start_time = time.time()
        if total_steps % trainOPT.print_freq == 0:
            t_data = iter_start_time - iter_data_time
        visualizer.reset()
        total_steps += trainOPT.batchSize
        epoch_iter += trainOPT.batchSize
        model.set_input(data)
        model.optimize_parameters()

        if total_steps % trainOPT.display_freq == 0:
            save_result = total_steps % trainOPT.update_html_freq == 0
            visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

        if total_steps % trainOPT.print_freq == 0:
            losses = model.get_current_losses()
            t = (time.time() - iter_start_time) / trainOPT.batchSize
            visualizer.print_current_losses(epoch, epoch_iter, losses, t, t_data)
            if trainOPT.display_id > 0:
                visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, trainOPT, losses)

            fn = open(log_filename,'a')        
            fn.write('\t{}'.format(epoch))
            fn.write('\n')
            fn.write(str(losses))
            fn.write('\n')
            fn.close()

        if total_steps % trainOPT.save_latest_freq == 0:
            print('saving the latest model (epoch %d, total_steps %d)' %
                  (epoch, total_steps))
            model.save_networks('latest')

        iter_data_time = time.time()
    if epoch % trainOPT.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' %
              (epoch, total_steps))
        model.save_networks('latest')
        model.save_networks(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' %
          (epoch, trainOPT.niter + trainOPT.niter_decay, time.time() - epoch_start_time))
    model.update_learning_rate()

Streaming output truncated to the last 5000 lines.
(epoch: 44, iters: 56100, time: 0.005, data: 0.003) D_A: 0.148 G_A: 1.680 cycle_A: 0.130 idt_A: 0.000 D_B: 0.002 G_B: 1.003 cycle_B: 0.461 idt_B: 0.000 
(epoch: 44, iters: 56200, time: 0.004, data: 0.003) D_A: 0.259 G_A: 0.067 cycle_A: 0.122 idt_A: 0.000 D_B: 0.002 G_B: 0.978 cycle_B: 0.549 idt_B: 0.000 
(epoch: 44, iters: 56300, time: 0.004, data: 0.003) D_A: 0.186 G_A: 1.835 cycle_A: 0.122 idt_A: 0.000 D_B: 0.002 G_B: 0.998 cycle_B: 0.584 idt_B: 0.000 
(epoch: 44, iters: 56400, time: 0.004, data: 0.003) D_A: 0.067 G_A: 0.615 cycle_A: 0.123 idt_A: 0.000 D_B: 0.002 G_B: 1.011 cycle_B: 0.558 idt_B: 0.000 
(epoch: 44, iters: 56500, time: 0.005, data: 0.003) D_A: 0.080 G_A: 0.513 cycle_A: 0.132 idt_A: 0.000 D_B: 0.002 G_B: 0.975 cycle_B: 0.554 idt_B: 0.000 
(epoch: 44, iters: 56600, time: 0.004, data: 0.003) D_A: 0.051 G_A: 0.950 cycle_A: 0.126 idt_A: 0.000 D_B: 0.002 G_B: 1.004 cycle_B: 0.570 idt_B: 0.000 
(epoch: 44, iters: 56700, time:

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-952b09e7bda9>", line 33, in <module>
    fn = open(log_filename,'a')
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Pollen-Adaptation/cyclegan/checkpoints/train.log'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number

FileNotFoundError: ignored